# Timing of oab
## Run these once, at the start of the notebook

In [ ]:
import logging
from time import perf_counter

import numpy as np
import pandas as pd
from rich import print

import oab

In [2]:
# make results list
results_table = []

## Personalize the following cell

In [3]:
# personalize this for running the notebook in different ways
algo_name = "DNN"

how_many_iterations = 2  # how many images each combination of the above? try: 100

## Run these after each "personalization"
To get results on multiple datasets, algorithms, classes

### Timed cells

In [ ]:
datasets = ["mnist", "fashion", "emnist"]
results = {}

for dataset_name in datasets:
    results[dataset_name] = {}

    (X_train, Y_train), (X_test, Y_test), (X_tree, Y_tree) = oab.get_data(dataset_name)
    my_dom = oab.Domain(dataset_name, algo_name)
    index = {}
    points = {}
    for my_class in range(10):
        index[my_class] = np.where(Y_test == my_class)[0]
    my_dom.load()
    my_index = 20  # to change with random

    for my_class in [int(x) for x in my_dom.classes]:
        explainers = []
        start = perf_counter()

        for i in range(how_many_iterations):
            testpoint = oab.TestPoint(X_test[my_index], my_dom)
            logging.info(f"start explanation of index {my_index}")
            exp = oab.Explainer(testpoint, howmany=5)

            if not exp.factuals:
                exp.factuals = []
            explainers.append(exp)
            print(f"# factuals made: {len(exp.factuals)}")

        end = perf_counter()

        # compute time
        execution_time = end - start
        # append into results_table
        results_table.append(
            {
                "Dataset": dataset_name,
                "Algo": algo_name,
                "Class": my_class,
                "Time": round(execution_time, 2),
            }
        )

        # flatten the explainers
        factuals = []
        for explainer in explainers:
            for point in explainer.factuals:
                factuals.append(point)

        # compute edges
        for dimensions in factuals[0]:
            print(dimensions.shape)
            pass

        # calculate area of hyperpol
        lengths = np.ndarray
        for dimension in dimensions:
            lengths.append(abs(dimension.min - dimension.max))
        area = lengths.prod()

        # report this area back as the density of a group of points.
        results[dataset_name][my_class] = area

        break  # for now, do it once.

## Run this to show the results table

In [ ]:
results_dataframe = pd.DataFrame.from_records(results_table)
results_dataframe